In [1]:
from binance.client import Client
from binance.enums import HistoricalKlinesType
from datetime import datetime, timedelta
import pandas as pd
import pandas_ta as ta
pd.set_option('display.max_rows', 500)
import requests,json
import plotly.graph_objs as go

ModuleNotFoundError: No module named 'binance.client'; 'binance' is not a package

In [555]:
client = Client('pKUTsuAWQU1DdpjAAxybUTLDHuSPQBSVivxsAESKn10toEpS3zvyWqMXP0s27Jp2', 'KmDujNaQ3LXyWRor8Y9qFI275xcXg6d5rNIdUpdLkI19ZxPj4sigPmIviyJx5n3j')

In [556]:
# Define column names for the DataFrame
colnames = [
    'time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
    'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
    'Taker buy quote asset volume', 'Ignore'
]

# Get all USDT pairs
BASE_URL = 'https://api.binance.com'
resp = requests.get(BASE_URL+'/api/v1/ticker/allBookTickers')
ticker_list = json.loads(resp.content)
symbols = [ticker['symbol'] for ticker in ticker_list if str(ticker['symbol'][-4:]) == 'USDT']


interval = '1d'  # Supports intervals like 1m, 1H, 1D, etc.
start = (datetime.today() - timedelta(days=90)).strftime('%m/%d/%Y 00:00:00')   # Start date
end = datetime.today().strftime('%m/%d/%Y 00:00:00') # End date

client = Client()  # Initialize your Binance API client


In [557]:
dicionario_adx = {
    '0-25': 'Tendência fraca ou inexistente',
    '25-50': 'Tendência forte',
    '50-75': 'Tendência muito forte',
    '75-100': 'Tendência extremamente forte'
}

def check_adx(value):
    for key, name in dicionario_adx.items():
        range_start, range_end = map(int, key.split('-'))
        if range_start <= value <= range_end:
            return name
    return None

In [558]:
def getting_data(symbol):
    print('Getting ' + symbol)

    # Fetch historical klines data
    klines_generator = client.get_historical_klines_generator(
        symbol=symbol,
        interval=interval,
        start_str=start,
        end_str=end,
        klines_type=HistoricalKlinesType.SPOT
    )
    klines_list = list(klines_generator)
    data = {
            'symbol': symbol,
            'data': [{
                'time_period_start': kline[0],
                'price_open': float(kline[1]),
                'price_high': float(kline[2]),
                'price_low': float(kline[3]),
                'price_close': float(kline[4]),
                'volume': float(kline[5])
            } for kline in klines_list]
        }

    dataframe = []
    symbol = data['symbol']
    dataframe = []
    for entry in data['data']:
        time = entry['time_period_start']
        prices = {
            'symbol': symbol,
            'time_period_start': time,
            'price_open': entry['price_open'],
            'price_high': entry['price_high'],
            'price_low': entry['price_low'],
            'price_close': entry['price_close'],
            'volume': entry['volume']
        }
        dataframe.append(prices)
            
    df = pd.DataFrame(dataframe)

    df['time_period_start'] = pd.to_datetime(df['time_period_start'], unit='ms')

    df[['price_open', 'price_high', 'price_low', 'price_close']] = df[['price_open', 'price_high', 'price_low', 'price_close']].map(lambda x: float(x))

    # Calculate ADX, DI+ and DI- manually
    df[['adx', 'dmp', 'dmn']] = ta.adx(df['price_high'], df['price_low'], df['price_close'], length = 14)
    df['adx_trend'] = df['adx'].apply(check_adx)
    
    # # Calculate ichimoku cloud  
    df[['leading_span_a', 'leading_span_b', 'conversion_line', 'base_line', 'lagging_span']] = ta.ichimoku(df['price_high'], df['price_low'], df['price_close'])[0]
    df[['macd', 'macd_hist', 'macd_signal']] = ta.macd(df['price_close']).map(lambda x: float(x))

    return df


In [559]:
# Concatenating dataframes

testing = []
for crypto in symbols:
    try:
        testing.append(getting_data(crypto))
    except:
        pass
    
df_concat = pd.concat(testing)

Getting BTCUSDT
Getting ETHUSDT
Getting BNBUSDT
Getting BCCUSDT
Getting NEOUSDT
Getting LTCUSDT
Getting QTUMUSDT
Getting ADAUSDT
Getting XRPUSDT
Getting EOSUSDT
Getting TUSDUSDT
Getting IOTAUSDT
Getting XLMUSDT
Getting ONTUSDT
Getting TRXUSDT
Getting ETCUSDT
Getting ICXUSDT
Getting VENUSDT
Getting NULSUSDT
Getting VETUSDT
Getting PAXUSDT
Getting BCHABCUSDT
Getting BCHSVUSDT
Getting USDCUSDT
Getting LINKUSDT
Getting WAVESUSDT
Getting BTTUSDT
Getting USDSUSDT
Getting ONGUSDT
Getting HOTUSDT
Getting ZILUSDT
Getting ZRXUSDT
Getting FETUSDT
Getting BATUSDT
Getting XMRUSDT
Getting ZECUSDT
Getting IOSTUSDT
Getting CELRUSDT
Getting DASHUSDT
Getting NANOUSDT
Getting OMGUSDT
Getting THETAUSDT
Getting ENJUSDT
Getting MITHUSDT
Getting MATICUSDT
Getting ATOMUSDT
Getting TFUELUSDT
Getting ONEUSDT
Getting FTMUSDT
Getting ALGOUSDT
Getting USDSBUSDT
Getting GTOUSDT
Getting ERDUSDT
Getting DOGEUSDT
Getting DUSKUSDT
Getting ANKRUSDT
Getting WINUSDT
Getting COSUSDT
Getting NPXSUSDT
Getting COCOSUSDT
Getti

/var/folders/b2/cg93_pyd467c2cvwmrzywvx80000gp/T/ipykernel_39905/356178114.py:10: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [567]:
today = datetime.today().date() - timedelta(days=1)
df_adj = df_concat.loc[(df_concat['adx'] > 25) & 
                (df_concat['time_period_start'].dt.date == today) &

                # Ichimoku com menos preço maior do que linha de conversão e linha de base
                (df_concat['price_low'] > df_concat['conversion_line']) &
                (df_concat['price_low'] > df_concat['base_line']) &

                ## MACD histograma maior do que 0 e signal maior do que macd
                # (df_concat['macd_hist'] > 0) &
                (df_concat['macd_signal'] >= df_concat['macd']) ]

In [568]:
df_adj[['symbol', 'time_period_start', 'price_open', 'volume', 'adx', 'adx_trend', 'macd', 'macd_hist', 'macd_signal']].sort_values(by='adx', ascending=False).reset_index()

,index,symbol,time_period_start,price_open,adx,adx_trend,volume,macd,macd_hist,macd_signal
0,89,OSMOUSDT,2023-12-02,0.6931,66.109658,Tendência muito forte,2.811777e+06,0.053190,-0.005620,0.058810
1,89,AVAXUSDT,2023-12-02,21.8600,62.599197,Tendência muito forte,2.115296e+06,1.801021,-0.215408,2.016429
2,89,SOLUSDT,2023-12-02,59.7400,57.264579,Tendência muito forte,4.898504e+06,4.172503,-0.439542,4.612045
3,71,FTTUSDT,2023-12-02,4.1074,49.980157,Tendência forte,8.612603e+06,0.509726,-0.049473,0.559200
4,89,IMXUSDT,2023-12-02,1.4420,49.710754,Tendência forte,1.176053e+07,0.120471,-0.015642,0.136113
5,89,FETUSDT,2023-12-02,0.5543,48.264389,Tendência forte,5.734604e+07,0.046134,-0.000868,0.047003
6,89,NEXOUSDT,2023-12-02,0.7780,47.236752,Tendência forte,7.224524e+05,0.023498,-0.001799,0.025298
7,89,MOVRUSDT,2023-12-02,6.7540,47.093356,Tendência forte,2.550951e+05,0.391607,-0.030001,0.421608
8,89,PYRUSDT,2023-12-02,6.6530,43.478480,Tendência forte,7.071857e+05,0.343223,-0.062521,0.405745
9,89,ARUSDT,2023-12-02,8.4190,43.138343,Tendência forte,7.847180e+05,0.418247,-0.054374,0.472621


In [569]:
df_concat.to_csv('crypto_data.csv', index=False)

In [562]:
# # Create the candlestick trace for price
# candlestick = go.Candlestick(
#     x=df_concat['time_period_start'],
#     open=df_concat['price_open'],
#     high=df_concat['price_high'],
#     low=df_concat['price_low'],
#     close=df_concat['price_close'],
#     name='Price'
# )

# # Create the figure for price (candlestick chart)
# fig = go.Figure(data=candlestick)

# # Update layout and add title
# fig.update_layout(
#     title='Price Candlestick Chart using Plotly',
#     xaxis_title='Date',
#     yaxis_title='Price'
# )

# # Show the plot
# fig.show()

In [563]:
# # Create the ADX trace
# adx_line = go.Scatter(
#     x=df_concat['time_period_start'],
#     y=df_concat['adx'],
#     mode='lines',
#     name='ADX',
#     line=dict(color='black')  # Color for ADX line
# )

# # Create the DI+ trace
# plus_di_line = go.Scatter(
#     x=df_concat['time_period_start'],
#     y=df_concat['dmp'],
#     mode='lines',
#     name='DI+',
#     line=dict(color='green')  # Color for DI+
# )

# # Create the DI- trace
# minus_di_line = go.Scatter(
#     x=df_concat['time_period_start'],
#     y=df_concat['dmn'],
#     mode='lines',
#     name='DI-',
#     line=dict(color='red')  # Color for DI-
# )
# # Create a fixed line for value 20 in light grey
# fixed_line = go.Scatter(
#     x=df_concat['time_period_start'],
#     y=[20] * len(df_concat),  # Creating a list of 20s to match the length of the DataFrame
#     mode='lines',
#     name='Fixed Line (20)',
#     line=dict(color='lightgrey', dash='dash')  # Color and style for the fixed line
# )

# # Create the figure for ADX chart
# fig = go.Figure(data=[adx_line, plus_di_line, minus_di_line, fixed_line])

# # Update layout and add title
# fig.update_layout(
#     title='ADX, DI+ and DI- Line Chart with Fixed Line at 20 using Plotly',
#     xaxis_title='Date',
#     yaxis_title='Values'
# )

# # Show the plot
# fig.show()

In [564]:
# # Create Ichimoku Cloud chart
# fig2 = go.Figure()

# # Add candlestick chart
# fig2.add_trace(go.Candlestick(x=df_concat['time_period_start'],
#                              open=df_concat['price_open'],
#                              high=df_concat['price_high'],
#                              low=df_concat['price_low'],
#                              close=df_concat['price_close'],
#                              name='Candlestick'))

# # Add Ichimoku Cloud lines
# fig2.add_trace(go.Scatter(x=df_concat['time_period_start'], y=df_concat['conversion_line'], mode='lines', name='Conversion Line', line=dict(color='blue')))
# fig2.add_trace(go.Scatter(x=df_concat['time_period_start'], y=df_concat['base_line'], mode='lines', name='Base Line', line=dict(color='red')))
# fig2.add_trace(go.Scatter(x=df_concat['time_period_start'], y=df_concat['leading_span_a'], mode='lines', name='Leading Span A'))
# fig2.add_trace(go.Scatter(x=df_concat['time_period_start'], y=df_concat['leading_span_b'], mode='lines', name='Leading Span B'))

# # Update layout
# fig2.update_layout(title='Price Candlestick with Ichimoku Cloud Indicators',
#                   xaxis_title='Date',
#                   yaxis_title='Price',
#                   showlegend=True)

# # Display fig2ure
# fig2.show()